In [1]:
import os
import pandas as pd
import numpy as np

folder_path = '../ssd_data/smartlog2019ssd'
file_extension = '.csv'

for month in range(1, 2):
    target_month = f'2019{str(month).zfill(2)}'
    dfs = []  # dfs 리스트 초기화
    for file_name in os.listdir(folder_path):
        if file_name.endswith(file_extension):
            date = file_name
            date = date.split('.')[0]  # 파일명에서 날짜 추출
            file_month = date[:6]  # 날짜에서 월 추출
            if target_month == file_month:
                file_path = os.path.join(folder_path, file_name)
                df = pd.read_csv(file_path)
                dfs.append(df)

    combined_df = pd.concat(dfs, ignore_index=True)
    combined_df_name = f'combined_df{str(month).zfill(2)}'
    globals()[combined_df_name] = combined_df

In [2]:
combined_df01.shape

(12667767, 105)

In [3]:
print(combined_df01.columns)

Index(['disk_id', 'ds', 'model', 'n_1', 'r_1', 'n_2', 'r_2', 'n_3', 'r_3',
       'n_4',
       ...
       'n_242', 'r_242', 'n_244', 'r_244', 'n_245', 'r_245', 'n_175', 'r_175',
       'n_232', 'r_232'],
      dtype='object', length=105)


In [4]:
combined_df01.iloc[:100]

,disk_id,ds,model,n_1,r_1,n_2,r_2,n_3,r_3,n_4,...,n_242,r_242,n_244,r_244,n_245,r_245,n_175,r_175,n_232,r_232
0,100,20190101,MB2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,99.0,4.377350e+09,100.0,0.0,100.0,65535.0,NaN,NaN,NaN,NaN
1,100034,20190101,MC1,100.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100035,20190101,MC1,100.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10005,20190101,MB1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,99.0,2.041073e+09,100.0,0.0,100.0,65535.0,NaN,NaN,NaN,NaN
4,100055,20190101,MC1,100.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,100865,20190101,MC1,100.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,100866,20190101,MA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,2.003095e+07,NaN,NaN,NaN,NaN,100.0,9.924693e+11,100.0,0.0
97,100873,20190101,MA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,5.967380e+05,NaN,NaN,NaN,NaN,100.0,9.923871e+11,100.0,0.0
98,100882,20190101,MA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,1.193470e+06,NaN,NaN,NaN,NaN,100.0,7.219790e+11,100.0,0.0


In [5]:
# combined_df04_sample = combined_df04.sample(n=1000000, random_state=42)

In [6]:
# combined_df04_sample.shape

In [7]:
# combined_df04_sample.to_csv('combined_df01_sample.csv', index=False)

In [8]:
#라벨링 하기 label data 불러오기
label = pd.read_csv('../ssd_data/ssd_failure_label.csv')
label

,model,failure_time,disk_id
0,MA2,2019-03-22 10:24:38,4711
1,MA2,2019-02-22 04:56:06,82064
2,MA2,2018-12-17 12:16:33,32311
3,MA2,2018-05-19 17:32:03,18316
4,MA2,2018-10-25 04:00:50,32466
...,...,...,...
16300,MC2,2019-09-30 09:09:55,15910
16301,MC2,2019-05-30 10:52:18,837
16302,MC2,2019-01-19 05:35:06,13236
16303,MC2,2019-07-18 23:14:27,277


In [9]:
#label data 전처리
label['failure_time']=label['failure_time'].str[:10].str.replace('-', '')
label['failure_time']=label['failure_time'].astype('int64')
label['failure_time'].dtypes

dtype('int64')

In [10]:
# 조인 수행
combined_df01 = combined_df01.merge(label, how='left', left_on=['model', 'disk_id', 'ds'],right_on=['model', 'disk_id', 'failure_time'])

# label 열 생성 및 라벨링 수행
combined_df01['label'] = np.where(combined_df01['failure_time'].notnull(), 1, 0)
del(combined_df01['failure_time'])

In [11]:
combined_df01.shape

(12667767, 106)

In [12]:
combined_df01[combined_df01['label']==1]

,disk_id,ds,model,n_1,r_1,n_2,r_2,n_3,r_3,n_4,...,r_242,n_244,r_244,n_245,r_245,n_175,r_175,n_232,r_232,label
66154,32240,20190101,MB1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.269325e+11,100.0,0.0,100.0,65535.0,NaN,NaN,NaN,NaN,1
285149,17909,20190101,MB1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.422788e+11,100.0,0.0,100.0,65535.0,NaN,NaN,NaN,NaN,1
328583,41741,20190101,MC1,100.0,1.000000e+00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
460660,51834,20190102,MC1,100.0,9.000000e+00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
534532,4788,20190102,MC2,100.0,4.000000e+00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11744427,21590,20190129,MA1,130.0,4.294967e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,100.0,1.035621e+12,NaN,NaN,1
11801917,2802,20190129,MA1,130.0,3.553202e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,100.0,1.018231e+12,NaN,NaN,1
11970225,33606,20190130,MA1,130.0,4.294967e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,100.0,6.489999e+11,NaN,NaN,1
12178478,57238,20190130,MA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.988705e+07,NaN,NaN,NaN,NaN,100.0,8.634548e+11,99.0,0.0,1


In [13]:
combined_df01.to_csv('combined_df01.csv', index=False)